In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

### Implement Linear Regression with Numpy only

In [2]:
# load dataset
data = '../data/cars.csv'
df = pd.read_csv(data)
df.head(2)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650


In [6]:
import wrangle as wr

In [7]:
df = wr.rename_columns(df)
df.columns

Index(['make', 'model', 'year', 'engine_fuel_type', 'engine_hp',
       'engine_cylinders', 'transmission_type', 'driven_wheels',
       'number_of_doors', 'market_category', 'vehicle_size', 'vehicle_style',
       'highway_mpg', 'city_mpg', 'popularity', 'msrp'],
      dtype='object')

__Train, validate, test split__

In [8]:
# make the split reproducible
np.random.seed(2912)

n = len(df)

# 20% for validate, 20% for test and 60% for train
n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)

# shuffle values to make the split random
idx = np.arange(n)
np.random.shuffle(idx)

# rearrange the values inside the dataframe
df_shuffled = df.iloc[idx]

# split with slicing the data
df_train = df_shuffled.iloc[:n_train].copy().reset_index(drop=True)
df_val = df_shuffled.iloc[n_train:n_train+n_val].copy().reset_index(drop=True)
df_test = df_shuffled.iloc[n_train+n_val:].copy().reset_index(drop=True)

In [9]:
df_train.head(2)

,make,model,year,engine_fuel_type,engine_hp,engine_cylinders,transmission_type,driven_wheels,number_of_doors,market_category,vehicle_size,vehicle_style,highway_mpg,city_mpg,popularity,msrp
0,mercedes-benz,350-class,1991,diesel,134.0,6.0,automatic,rear_wheel_drive,4.0,"diesel,luxury",large,sedan,23,19,617,2178
1,aston_martin,db9,2014,premium_unleaded_(required),510.0,12.0,automatic,rear_wheel_drive,2.0,"exotic,high-performance",midsize,coupe,19,13,259,185800


In [5]:
# check if all element are included in the split
len(df) == len(df_train) + len(df_val) + len(df_test)

True

In [11]:
# separate the target variable and make its logarithmic transformation
y_train = np.log1p(df_train.msrp.values)
y_val = np.log1p(df_val.msrp.values)
y_test = np.log1p(df_test.msrp.values)

# drop the target variable from the data sets
del df_train['msrp']
del df_val['msrp']
del df_test['msrp']

### Linear Regression

For one single $y$ the formula is $g(x_i) = w_0 + \sum_{j=0}^{n-1} w_j * x_{ij}$, where:
- $w_0$ is bias (if we don't know any other features, the final result will be equal to this number), or `y-intersect` on the graph. 
- $n$ - number of features (we count from 0 like in array, that's why $j=0$ and the count goes till $n-1$, normally we count from 1 till $n$). 
- $w_j$ weight of the $j$ -th feature
- $x_{ij}$ the $j$ -th feature of $i$ -th row

In [22]:
# make up values
xi = [453, 11, 86]
w0 = 5.45 # bias
w = [0.04, -0.1, 0.01] # weights

In [23]:
def linear_regression_1(xi):
    n = len(xi)

    pred = w0
    for j in range(n):
        pred = pred + w[j]*xi[j]
    return pred

In [28]:
pred = linear_regression_1(xi)
pred

23.33

In [25]:
# convert the prediction value (log(y+1)) to price
price = np.expm1(pred)
display(price.round(2))
price == np.exp(pred) - 1

13554711010.88

True

#### Linear regression through vector - vector dot product

In [31]:
def dot(xi, w):
    n = len(xi)
    res = 0
    for j in range(n):
        res = res + xi[j] * w[j]
    return res 

def linear_regression_2(xi):
    return w0 + dot(xi, w)
    # same as
    #return w0 + xi.dot(w)

In [32]:
linear_regression_2(xi)

23.33

In [30]:
w0 + np.array(xi).dot(w)

23.33

We can avoid summing the vector multiplication with $w_0$ by adding $w_0$ in the beginning of the weights array `w` and `1` in the beginning of the features array `xi`

In [46]:
print([w0] + w)
# in numpy
np.insert(w, 0, w0 )

[5.45, 0.04, -0.1, 0.01]


array([ 5.45,  0.04, -0.1 ,  0.01])

In [34]:
[1] + xi

[1, 453, 11, 86]

In [47]:
# np.insert(array_to_insert_to, position, values, axis(for 2D))
w = np.insert(w, 0, w0)
xi = np.insert(xi, 0, 1)

In [48]:
print(w)
print(xi)

[ 5.45  0.04 -0.1   0.01]
[  1 453  11  86]


In [49]:
# make up values
xi = [453, 11, 86]
w0 = 5.45 # bias
w = [0.04, -0.1, 0.01] # weights

In [50]:
def linear_regression_3(xi, w, w0):
    wn = np.insert(w, 0, w0)
    xn = np.insert(xi, 0, 1)
    return xn.dot(wn)


In [51]:
linear_regression_3(xi, w, w0)

23.33

#### Linear regression as matrix-vector dot product

$X * w = y$, where:
- `X` -> matrix with arrays of `n` features with leading 1 as the 1st element
- `w` -> vector(array) with weight with bias `w0` as the st element
- `y` -> vector(array) of predictions

In [57]:
# features matrix
X = np.array([
    [453, 11, 86],
    [365, 20, 76],
    [501, 16, 70],
    [467, 19, 66],
    [398, 21, 72],
    [403, 13, 65],
    [600, 23, 93],
    [388, 14, 80],
    [345, 12, 98]
])
w0 = 5.45 # bias
w = [0.04, -0.1, 0.01] # weights

Dot multiplication matrix `X` with weights `w` -> returns an array of predictions.

`X.dot(w) = y_pred`

In [53]:
# add ones to X
np.insert(X, 0, np.ones(len(X)), axis = 1)

array([[  1, 453,  11,  86],
       [  1, 365,  20,  76],
       [  1, 501,  16,  70],
       [  1, 467,  19,  66],
       [  1, 398,  21,  72],
       [  1, 403,  13,  65],
       [  1, 600,  23,  93],
       [  1, 388,  14,  80],
       [  1, 345,  12,  98]])

In [54]:
# add w0 to w
np.insert(w, 0, w0)

array([ 5.45,  0.04, -0.1 ,  0.01])

In [68]:
X = np.insert(X, 0, np.ones(len(X)), axis = 1)
w = np.insert(w, 0, w0)
# calculate predictionss
X.dot(w)

array([23.33, 18.81, 24.59, 22.89, 19.99, 20.92, 28.08, 20.37, 19.03])

#### Train regression model and calculate weights

`X` -> matrix with dimension `(m, n+1)`, where:
- `m` -> number of rows, 
- `n` -> number of features,
- `n+1` -> adding 1 in the beginning of every feature vector (see above)

We can not create an inverse matrix of `X` because its not squared. If the inverse matrix of `X` existed, we could solve for `w` by multiplying both sides with an inversed matrix: 
- $X^{-1} * X * w = X^{-1} * y$, 
- then $X^{-1} * X$ turnes into identity matrix: $I * w = X^{-1} * y$. 
- As we know from linear algebra, identity matrix multiplied by vector equals that vector $w = X^{-1} * y$.

However can make matrix squared by using dot multiplication with its transposed version, this calls __gram matrix__. $X^T * X * w = X^T * y$, it returns matrix with the shape `(n+1, n+1)`.

In [69]:
X

array([[  1, 453,  11,  86],
       [  1, 365,  20,  76],
       [  1, 501,  16,  70],
       [  1, 467,  19,  66],
       [  1, 398,  21,  72],
       [  1, 403,  13,  65],
       [  1, 600,  23,  93],
       [  1, 388,  14,  80],
       [  1, 345,  12,  98]])

In [67]:
X.shape # m (rows) = 9, n (features) = 3, n+1 = 4

(9, 4)

#### Step 1.
Multiply by sides by transposed matrix `X1.T`

$X^T * X * w = X^T * y$, `y` -> `y_train`

In [70]:
# gram matrix, shape (4, 4)
X.T.dot(X)

array([[      9,    3920,     149,     706],
       [   3920, 1757906,   66141,  308091],
       [    149,   66141,    2617,   11632],
       [    706,  308091,   11632,   56490]])

#### Step 2.
Multiply both sides by __inversed__ gram matrix.

$(X^T * X)^{-1} * X^T * X * w = (X^T * X)^{-1} * X^T * y$

This will turn $(X^T * X)^{-1} * X^T * X$ into __identity matrix__.

In [77]:
gram = X.T.dot(X)
np.linalg.inv(gram).round(2)

array([[ 9.72, -0.01, -0.09, -0.07],
       [-0.01,  0.  , -0.  , -0.  ],
       [-0.09, -0.  ,  0.01,  0.  ],
       [-0.07, -0.  ,  0.  ,  0.  ]])

In [78]:
# get identity matrix
gram.dot(np.linalg.inv(gram)).round(1)

array([[ 1.,  0., -0.,  0.],
       [-0.,  1., -0.,  0.],
       [ 0., -0.,  1.,  0.],
       [-0.,  0., -0.,  1.]])

#### Step 3
Solve for `w`:

$w = (X^T * X)^{-1} * X^T * y$

The first element of `w`, `w[0]` is a bias, the rest are weights.
 
$Predictions = w_0 + w_1 * x_{i1} + w_2 * x_{i2} + ... + w_n * x_{in}$

In [ ]:
def get_weights(X_train: np.array, y_train):
    '''
    Parameters:
        X_train: 2-D array of features
        y_train: 1-D array of target variable
    The function calculates weights for linear regression equation.
    Returns:
        w[0] -> float, bias (y-intersect)
        w[1:] -> array of weights (floats)
    '''
    # add 1 to the beginning of every vector in features
    X = np.insert(X, 0, np.ones(len(X)), axis = 1)
    # get gram matrix
    XTX = X.T.dot(X)
    # inverse XTX
    XTX_inv = np.linalg.inv(XTX)
    # calculate weights
    w = XTX_inv.dot(X.T).dot(y)
    bias = w[0]
    weights = w[1:]

    return bias, weights